In [1]:
import shared.config as config
import pymongo
import gridfs
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan
import ast
#import attributes


import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
from sqlalchemy.types import VARCHAR

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

from sqlalchemy import and_, or_

from PIL import Image, ExifTags
import magic
import io
import shared.image_util as iu
import os

from sqlalchemy.dialects import postgresql
from sqlalchemy.schema import CreateTable
from sqlalchemy import Table, Column, String, MetaData

from models import Stelling, Project, Vondst

ModuleNotFoundError: No module named 'flask'

In [3]:
def AggregateData(df, fase): 
    df['teller'] = pd.to_numeric(df['teller'])
    df = df.groupby('soort').agg({'project':lambda x: list(x), 'teller':lambda x: sum(x)})
    df['Count'] = df.apply(lambda x: len(x.project), axis=1)
    df['Stage'] = fase
    df = df.sort_values('teller', ascending=False).reset_index()
    df = df.rename(columns={'soort': 'Soort', 'project': 'Projecten', 'teller': 'Aantal_Records', 'Count': 'Aantal_projecten'}).sort_index()
    return df

def getData(collection, soort): 
    df1 = pd.DataFrame(list(collection.aggregate(grp_aggr_artef)))
    df2 = pd.DataFrame(list(collection.aggregate(grp_aggr)))

    return AggregateData(df1.append(df2[df2.soort != 'Artefact']), soort)

In [4]:
lst_tables = ['Def_Project', 'Def_Put', 'Def_Vondst', 'Def_Spoor', 'Def_Stelling', 'Def_Doos', 'Def_Standplaats', 'Def_Plaatsing', 'Def_Vlak', 'Def_Vindplaats', 'Def_Artefact'] #['Def_Project', 'Def_Vondst', 'Def_Stelling', 'Def_Plaatsing', 'Def_Vindplaats', 'Def_Artefact']
regexProject = re.compile(r'\'project\': \'(.*?)\'') # regex to replace Object

def getProject(brondata):    
    #print(brondata)
    if brondata is None or brondata != "":
        return regexProject.search(brondata).group(1)
    else: 
        return "" 

df_tables_projects = pd.DataFrame()
    
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    for table in lst_tables:     
        df = pd.read_sql_query('SELECT brondata from "' + table + '"', connection)
        
        df_out = pd.DataFrame()
        df_out['project'] = df.apply(lambda x: getProject(x['brondata']), axis=1)
        df_out['soort'] = table[4:]
        df_tables_projects = pd.concat([df_tables_projects, pd.DataFrame(df_out).groupby(['project', 'soort']).size().reset_index(name='teller')])

with engine.connect() as connection:
    df = pd.read_sql_query('SELECT brondata, artefactsoort from "Def_Artefact"', connection)

    df_out = pd.DataFrame()
    df_out['project'] = df.apply(lambda x: getProject(x['brondata']), axis=1)
    df_out['soort'] = df['artefactsoort']
    df_tables_projects = pd.concat([df_tables_projects, pd.DataFrame(df_out).groupby(['project', 'soort']).size().reset_index(name='teller')])

        
df_doelsysteem = AggregateData(df_tables_projects, 'Doelsysteem')
df_doelsysteem.head(5)

,Soort,Projecten,Aantal_Records,Aantal_projecten,Stage
0,Artefact,"[0, AANTAL, AANW, AFMETING, ARTEFACT, ASSOCIA,...",280571,266,Doelsysteem
1,Aardewerk,"[AANTAL, ARTEFACT, BAKSEL, CAT, DAT_BEG, DAT_E...",72533,81,Doelsysteem
2,Dierlijk_Bot,"[AANTAL, ARTEFACT, ASSOCIA, BEWERK, BRAND, DB0...",51351,52,Doelsysteem
3,Onbekend,"[AANTAL, ARTEFACT, DC160, GEWICHT, OPMERKING, ...",30054,6,Doelsysteem
4,Vondst,"[DB001, DB002, DB003, DB004, DB005, DB006, DB0...",29042,112,Doelsysteem


In [23]:
from sqlalchemy import create_mock_engine

def metadata_dump(sql, *multiparams, **params):
    # print or write to log or file etc
    print(sql.compile(dialect=postgresql.dialect()))

engine = create_mock_engine("postgresql://", metadata_dump)
table_sql = CreateTable(Project.__table__).compile(engine)

print(table_sql)
MetaData.create_all(engine, checkfirst=False) 


CREATE TABLE "Def_Project" (
	primary_key SERIAL NOT NULL, 
	projectcd VARCHAR(12) NOT NULL, 
	projectnaam VARCHAR(1024), 
	jaar INTEGER, 
	toponiem VARCHAR(1024), 
	trefwoorden VARCHAR(1024), 
	xcoor_rd FLOAT, 
	ycoor_rd FLOAT, 
	longitude FLOAT, 
	latitude FLOAT, 
	herkomst TEXT, 
	soort VARCHAR(80), 
	brondata TEXT, 
	_id VARCHAR, 
	key TEXT, 
	aantal_artefacten INTEGER, 
	PRIMARY KEY (primary_key)
)




AttributeError: 'MockConnection' object has no attribute 'bind'